In [1]:
# Imports
import pandas as pd
import numpy as np
import pickle
from stqdm import stqdm

# PDF
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams

import pdfplumber
import re
import io

# Summarization using extractive bert
from summarizer import Summarizer, sentence_handler
#import tensorflow_hub as hub

# BERT based models for document search
from sentence_transformers import SentenceTransformer

C:\Users\adshafi\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def load_models():   
    qa = SentenceTransformer('sentence-transformers/multi-qa-distilbert-dot-v1')
    summ = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True)
    return qa, summ


def load_pdf(file,n=0)->str:
    
    if isinstance(file, str):
        fp = open(file, 'rb')
    else: 
        fp = file
        
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # Process each page contained in the document.
    for i, page in enumerate(PDFPage.get_pages(fp)):
        if i+1 > n:
            interpreter.process_page(page)
    text = retstr.getvalue()
    return text

def get_articles(text:str)->pd.DataFrame:
    
    data = text.split('\x0c')
    
    df = pd.DataFrame(enumerate(data,1),columns=['page','text'])
    
    df = df.assign(text=df['text'].str.split('\n\n')).explode('text').reset_index(drop=True)
    df['text'] = df['text'].str.strip().str.replace('\s+',' ')
    df['words'] = df['text'].apply(lambda x: len(x.split(' ')))
    df = df.loc[lambda x: x.text.astype(bool)].loc[lambda x: x.words>20].drop_duplicates(subset='text')
    df.drop('words',axis=1,inplace=True)

In [68]:
qa, summ = load_models()

2022-01-25 14:20:05.250 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: sentence-transformers/multi-qa-distilbert-dot-v1
2022-01-25 14:20:05.269 INFO    sentence_transformers.SentenceTransformer: Did not find folder sentence-transformers/multi-qa-distilbert-dot-v1
2022-01-25 14:20:05.270 INFO    sentence_transformers.SentenceTransformer: Try to download model from server: https://sbert.net/models/sentence-transformers/multi-qa-distilbert-dot-v1.zip
2022-01-25 14:20:05.276 INFO    sentence_transformers.SentenceTransformer: Downloading sentence transformer model from https://sbert.net/models/sentence-transformers/multi-qa-distilbert-dot-v1.zip and saving it at C:\Users\adshafi/.cache\torch\sentence_transformers\sbert.net_models_sentence-transformers_multi-qa-distilbert-dot-v1
Exception when trying to download https://sbert.net/models/sentence-transformers/multi-qa-distilbert-dot-v1.zip. Response 404
2022-01-25 14:20:06.383 WARNING sentence_transfor

2022-01-25 14:20:07.322 INFO    filelock: Lock 1589270693488 released on C:\Users\adshafi/.cache\huggingface\transformers\c8c7397094cde09588239e44541b09710ef9a899601c456f6fb8e2a44be94850.0c4b40a43fbbe2d04ddc2ff312b2cd13c8056078f520bb379117b25432d7c5fe.lock


2022-01-25 14:20:07.756 INFO    filelock: Lock 1589315941568 acquired on C:\Users\adshafi/.cache\huggingface\transformers\6e9e53c09b95f0ae1d36e8b36cf61de82d2294cd4c3912c6d25fafd05aff6e28.c1128180e82193c9bb4a2c1992d7107968aa54104c03a5d53ed92eb334203090.lock


2022-01-25 14:21:00.781 INFO    filelock: Lock 1589315941568 released on C:\Users\adshafi/.cache\huggingface\transformers\6e9e53c09b95f0ae1d36e8b36cf61de82d2294cd4c3912c6d25fafd05aff6e28.c1128180e82193c9bb4a2c1992d7107968aa54104c03a5d53ed92eb334203090.lock


2022-01-25 14:21:03.172 INFO    filelock: Lock 1589315253488 acquired on C:\Users\adshafi/.cache\huggingface\transformers\399726f9a9f5b3f46d502832a2c5ea35ce4236c1e9256c8d783eafcc2cbf8822.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


...

2022-01-25 14:21:09.345 INFO    filelock: Lock 1589315252624 released on C:\Users\adshafi/.cache\huggingface\transformers\c71dc1067871f671af8994af9e1730eea4ef810ba8ef7cf236152e984af745e3.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4.lock
2022-01-25 14:21:09.433 INFO    sentence_transformers.SentenceTransformer: Use pytorch device: cpu


2022-01-25 14:21:09.821 INFO    filelock: Lock 1589316090416 acquired on C:\Users\adshafi/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock


2022-01-25 14:21:10.266 INFO    filelock: Lock 1589316090416 released on C:\Users\adshafi/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock


2022-01-25 14:21:10.640 INFO    filelock: Lock 1589315096384 acquired on C:\Users\adshafi/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


2022-01-25 14:22:02.672 INFO    filelock: Lock 1589315096384 released on C:\Users\adshafi/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


In [128]:
with pdfplumber.open('../pip_guide_2.pdf') as pdf:
    pdf_data = ' '.join([i.extract_text() for i in pdf.pages[3:]])

In [129]:
text = load_pdf('../pip_guide_2.pdf', 3)

In [127]:
def pdf_data_df(text):
    reg = re.compile(r'[\d]+[\.][\d]+[\.][\d]+')
    chapter_list = reg.findall(text)
    
    chapters = []
    for i in range(len(chapter_list)-1):
        chapters.append(text.split(chapter_list[i])[1].split(chapter_list[i+1])[0])

    chapters.append(text.split(chapter_list[-1])[1].split('2.3')[0])
    
    a = pd.DataFrame(zip(chapter_list,chapters),columns=['chapter','text'])
    a['text'] = a['text'].str.replace('\n',' ').str.replace('\x0c', '').str.strip()
    
    return a

In [136]:
test = pdf_data_df(text)
a = pdf_data_df(pdf_data)

In [133]:
test[test['chapter']=='2.2.8']

,chapter,text
38,2.2.8,


In [135]:
test2[test2['chapter']=='2.2.8']

,chapter,text
38,2.2.8,"As made clear in legislation, harm is in relat..."


In [150]:
clean_paras = pd.read_csv('paragraphs_clean.csv').drop(['original'], axis=1)
c = clean_paras['clean'].str.replace('\n', ' ').str.strip().apply(qa.encode).apply(pd.Series)


In [151]:
def ask(q:str, X:pd.DataFrame, s:pd.DataFrame, n: int, model)->pd.Series:
    
    embedding = np.array(model.encode([q])[0])
        
    sorted_index = (X
                    .apply(lambda row: np.dot(row, embedding), axis=1)
                    .abs()
                    .sort_values(ascending=False)
                   )
    
    return s.loc[sorted_index.index].head(n)

In [154]:
ans = ask('preparing food activity', X=c, s=clean_paras, n=3, model=qa)

In [155]:
ans

,chapter,clean
75,2.2.29,Worked Example 2 (see 2.2.28):\n\nIn the Prepa...
74,2.2.28,Worked Example 3:\n\nMs Z can prepare and cook...
62,2.2.16,Where the act of completing the activity means...


In [156]:
a.to_csv('paragraphs.csv', index=False)
c.to_csv('paragraphs_embedded.csv', index=False)